# Function Calling
Function Calling은 기본적으로 챗봇이 사용자에게 **어떤 기능(function)을 실행하기 위한 파라미터**를 계속 요청하는 기법을 뜻합니다.

**예시**
- 사용자에게 id를 입력 받아 주문 배송 일자를 리턴해주는 챗봇
- 사용자에게 책상의 가로,세로,깊이를 입력 받아 상품을 추천해주는 챗봇

사용자로부터 기능을 실행하기 위한 입력값을 받기 위해 챗봇은 끊임없이 질문을 던져야 합니다.

이 때 주의해야 할 점은 챗봇이 직접 개발자가 원하는 함수를 호출하는 것이 아니라는 것입니다. **⭐️단순히 어떤 함수가 호출 될 준비가 되었다⭐️**라는 것만 알려줍니다. 즉 함수 호출은 개발자가 **직접**해야 한다는 것입니다.

In [121]:
import os
import sys
import json
from datetime import datetime
import openai

sys.path.append(os.path.abspath("../../data/API_KEY"))

In [122]:
from openai import OpenAI
import OpenAI_API_Key


API_KEY = OpenAI_API_Key.API_KEY

client = OpenAI(api_key=API_KEY)

print(openai.__version__)

1.51.2


In [123]:
# get str type input from server, return type = datatime
def get_delivery_date(order_id: str) -> datetime:
    # code for DB access comes here
    # return DB access result
    return datetime.today().strftime("%Y-%m-%d")

In [124]:
def get_customer_name(c_name: str) -> str:
    return c_name

In [125]:
# LLM 에서 사용할 함수 tool 생성
delivery_tool = {
    "type": "function",  # 도구의 타입을 function 으로 설정해 함수 호출 기능 제공을 명시
    "function": {
        "name": "get_delivery_date",  # 함수의 이름 정의
        # 챗봇이 명확하게 알 수 있도록 함수의 목적과 사용 사례 설명
        "description": "고객의 주문에 대한 배송 날짜를 확인합니다. 예를 들어, 고객이 '내 패키지가 어디에 있나요?'라고 물을 때 이 함수를 호출하세요.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {"type": "string", "description": "고객의 주문 id"}
            },
            "required": ["order_id"],  # 함수 호출 시 반드시 제공되어야 하는 필수 인자
            "additionalProperties": False,  # 추가적인 인자 허용 x
        },
    },
}

parse_name_tool = {
    "type": "function",
    "function": {
        "name": "get_customer_name",
        "description": "고객의 이름을 확인합니다. 예를 들어, 고객이 '내 패키지가 어디에 있나요?'라고 물을 때 이 함수를 호출하세요. ",
        "parameters": {
            "type": "object",
            "properties": {"c_name": {"type": "string", "description": "고객의 이름"}},
            "required": ["c_name"],
            "additionalProperties": False,
        },
    },
}

tools = [delivery_tool, parse_name_tool]

In [133]:
messages = []
messages.append(
    {
        "role": "system",
        "content": "당신은 도움이 되는 고객 지원 어시스턴트입니다. 제공된 도구를 사용하여 사용자를 지원하세요.",
    }
)
messages.append(
    {
        "role": "user",
        "content": "안녕하세요, 제 이름은 John 입니다. 주문의 배송 날짜를 알려주실 수 있나요?",
    }
)
messages.append(
    {
        "role": "assistant",
        "content": "안녕하세요! 제가 도와드릴 수 있습니다. 주문 ID를 알려주시겠어요?",
    }
)
messages.append({"role": "user", "content": "제 생각엔 order12345인 것 같아요"})
# messages.append({"role": "user", "content": "쓸데없는 소리"}) => 출력 확인해보자

response = client.chat.completions.create(
    model="gpt-4o-mini", messages=messages, tools=tools  # 지원 도구 설정
)

response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_QG13dtwEEGqK9ry8yONwKgvC', function=Function(arguments='{"c_name": "John"}', name='get_customer_name'), type='function'), ChatCompletionMessageToolCall(id='call_oUCpUmbtIhnH5qHPqWpy2VNR', function=Function(arguments='{"order_id": "order12345"}', name='get_delivery_date'), type='function')])

In [127]:
# get_delivery_date() 를 위한 매개변수 추출
tool_call = response.choices[0].message.tool_calls

if tool_call:
    print(tool_call[0])
else:
    print(
        "함수를 호출하기 위한 작업이 마무리되지 않았습니다"
    )  # LLM 이 계속 재질문하기 때문에 필수 구현은 아님

ChatCompletionMessageToolCall(id='call_ySLXiwJMoEGquexI7Tev04hk', function=Function(arguments='{"c_name": "John"}', name='get_customer_name'), type='function')


In [128]:
arguments = []
arguments.append(json.loads(tool_call[0].function.arguments))
arguments.append(json.loads(tool_call[1].function.arguments))
arguments

[{'c_name': 'John'}, {'order_id': 'order12345'}]

In [129]:
name = get_customer_name(**arguments[0])
name

'John'

In [130]:
date = get_delivery_date(**arguments[1])
date

'2024-10-15'

In [131]:
print(f"{name} ordered this product on {date}")

John ordered this product on 2024-10-15


In [132]:
if response.choices[0].finish_reason == "tool_calls":
    arguments = json.loads(response.choices[0].message.tool_calls[1].function.arguments)
    delivery_date = get_delivery_date(**arguments)

    print(delivery_date)  # 이 내용을 Web UI 같은걸로 보여줄 수 있도록 코딩합니다.

2024-10-15
